In [12]:
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import treebank
from abc import abstractmethod
from nltk import sent_tokenize
import torch.nn as nn
import pandas as pd
import numpy as np
import collections
import itertools
import argparse
import easydict
import random
import pickle
import MeCab
import torch
import json
import sys
import os
import re

In [47]:
collections.Counter(itertools.chain.from_iterable(treebank.sents()))

Counter({'Pierre': 1,
         'Vinken': 2,
         ',': 4885,
         '61': 5,
         'years': 115,
         'old': 24,
         'will': 281,
         'join': 4,
         'the': 4045,
         'board': 30,
         'as': 385,
         'a': 1878,
         'nonexecutive': 5,
         'director': 32,
         'Nov.': 24,
         '29': 5,
         '.': 3828,
         'Mr.': 375,
         'is': 671,
         'chairman': 45,
         'of': 2319,
         'Elsevier': 1,
         'N.V.': 3,
         'Dutch': 3,
         'publishing': 13,
         'group': 43,
         'Rudolph': 3,
         'Agnew': 1,
         '55': 10,
         'and': 1511,
         'former': 19,
         'Consolidated': 2,
         'Gold': 2,
         'Fields': 2,
         'PLC': 13,
         'was': 367,
         'named': 22,
         '*-1': 1123,
         'this': 184,
         'British': 11,
         'industrial': 18,
         'conglomerate': 3,
         'A': 110,
         'form': 17,
         'asbestos': 11,
       

In [2]:
def config_parser(args):
    print('file path is ' + str(args.file_path))
    with open(args.config_path, 'rb') as f:
        config = easydict.EasyDict(json.load(f))
    config.model = args.model
    config.file_path = args.file_path
    config.dataset_path = args.dataset_path
    config.device = torch.device(args.device)
    return config

args = argparse.ArgumentParser(description='nlp embedding')
args.add_argument('-m', '--model', default='neg-sampling', type=str,
                  help='which model to use')
args.add_argument('-cp', '--config-path', default='config.json', type=str,
                  help='config file path (default: None)')
args.add_argument('-fp', '--file-path', default='D:\\data\\text\\news-articles\\kbanker_articles_subtitles.csv', type=str,
                  help='path to latest checkpoint (default: None)')
args.add_argument('-dp', '--dataset-path', default='data\\nlp_dataset.pkl', type=str,
                  help='if there is a pickled dataset')
args.add_argument('-d', '--device', default='cuda:0', type=str,
                  help='indices of GPUs to enable (default: all)')
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
config = config_parser(args.parse_args())
config['neg_sample_size'] = 5
config['file_path'] = 'D:\\data\\text\\torch-dataset\\kbanker_articles_processed.pkl'
config['file_path'] = 'E:\\data\\text\\news-articles\\kbanker_articles_subtitles.csv'
config['file_path'] = 'treebank'
config['model'] = 'fast-text'

file path is C:\Users\sylim2357\AppData\Roaming\jupyter\runtime\kernel-5eba494d-85f6-492a-8d9c-86555979d69c.json


In [3]:
config

{'batch_size': 128,
 'epoch': 20,
 'gpu': 0,
 'window_size': 2,
 'embed_dim': 20,
 'h_dim': 500,
 'neg_sample_size': 5,
 'model': 'fast-text',
 'file_path': 'treebank',
 'dataset_path': 'data\\nlp_dataset.pkl',
 'device': device(type='cuda', index=0)}

In [4]:
#pre-processing shit
def pre_process_raw_article(article):
    """Pre-processing news articles.
    
    Args
        article (str): article text
    
    """
    replacements = [
        ('[“”]', '"'),
        ('[‘’]', '\''),
        ('\([^)]*\)', ''),
        ('[^가-힣\'"A-Za-z0-9.\s\?\!]', ' '),
        ('(?=[^0-9])\.(?=[^0-9])', '. '),
        ('\s\s+', ' ')
    ]
    
    for old, new in replacements:
        article = re.sub(old, new, article)
        
    return article

def mecab_tokenize(sentence):
    t = MeCab.Tagger()
    return [re.split(',', re.sub('\t', ',', s))[0] for s in t.parse(sentence).split('\n') if (s!='') & ('EOS' not in s)]

In [6]:
def pre_process(articles):
    print('preprocessing the corpus')
    articles = [pre_process_raw_article(article) for article in articles]
    sentences = itertools.chain.from_iterable([sent_tokenize(article) for article in articles])
    corpus = [mecab_tokenize(s) for s in list(sentences)]
    return corpus

kbanker_processed = pre_process(pd.read_csv(config.file_path, encoding='utf-8')['article'].dropna().values)

preprocessing the corpus


In [15]:
with open('kbanker_articles_processed.pkl', 'wb') as f:
    pickle.dump(kbanker_processed, f)

In [18]:
def pre_process(articles):
    print('preprocessing the corpus')
    articles = [pre_process_raw_article(article) for article in articles]
    sentences = itertools.chain.from_iterable([sent_tokenize(article) for article in articles])
    corpus = [mecab_tokenize(s) for s in list(sentences)]
    corpus = [[ngram(w, 3) for w in s] for s in corpus]

    return corpus

def ngram(w, n):
    word = '<' + w + '>'
    if len(word) <= 3:
        return [word]
    else:
        ngram = []
        for i in range(n, len(word)+1):
            ngram += [word[i-n:i]]

        return ngram + [word]

articles = pd.read_csv(config.file_path, encoding='utf-8')['article'].dropna().values
corpus = pre_process(articles)

preprocessing the corpus


In [19]:
with open('D:\\data\\text\\torch-dataset\\kbanker_articles_processed_fasttext.pkl', 'wb') as f:
    pickle.dump(corpus, f)

In [5]:
class W2VDataset(Dataset):
    """W2V Dataset

    Args:
        config (dict): hyperparameters

    Attributes:
        root_dir (str): root
        word_to_idx (dict): word_to_idx mapping
        idx_to_word (dict): idx_to_word mapping
        x (list): train data (5-gram)
        y (list): label

    """

    def __init__(self, config):
        if 'pkl' in config.file_path:
            with open(config.file_path, 'rb') as f:
                corpus = pickle.load(f)[:1000]
        elif config.file_path == 'treebank':
            corpus = treebank.sents()
        else:
            articles = pd.read_csv(config.file_path, encoding='utf-8')['article'].dropna().values

            #pre process
            corpus = self.pre_process(articles)

        #construct word matrix
        self.word_to_idx, self.idx_to_word = self.construct_word_idx(corpus)
        corpus = [[self.word_to_idx[word] for word in sentence] for sentence in corpus]
        #make dataset
        self.x, self.y = self.construct_dataset(corpus, config)

    def pre_process(self, articles):
        print('preprocessing the corpus')
        articles = [pre_process_raw_article(article) for article in articles]
        sentences = itertools.chain.from_iterable([sent_tokenize(article) for article in articles])
        corpus = [mecab_tokenize(s) for s in list(sentences)]
        
        return corpus

    def construct_word_idx(self, corpus):
        print('constructing word matrix')
        word_set = set(itertools.chain.from_iterable(corpus))
        word_to_idx = {word : idx for idx, word in enumerate(word_set)}
        idx_to_word = {word_to_idx[word] : word for word in word_to_idx}
        
        return word_to_idx, idx_to_word

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.tolist()
        
        return self.x[idx], self.y[idx]

    def fast_text_pre_process(self, corpus):
        pass
    
    @abstractmethod
    def construct_dataset(self, corpus, config):
        raise NotImplementedError
        
class CBOWDataset(W2VDataset):
    """CBOW Dataset"""

    def __init__(self, config):
        super().__init__(config)

    def construct_dataset(self, corpus, config):
        print('constructing training dataset')
        x, y = [], []
        for sentence in corpus:
            for i in range(config.window_size, len(sentence) - config.window_size):
                x += [sentence[i-config.window_size:i] + sentence[i+1:i+condig.window_size+1]]
                y += [sentence[i]]
        
        return x, y

class SkipGramDataset(W2VDataset):
    """Skip-Gram Dataset"""

    def __init__(self, config):
        super().__init__(config)

    def construct_dataset(self, corpus, config):
        print('constructing training dataset')
        x, y = [], []
        for sentence in corpus:
            for i in range(config.window_size, len(sentence) - config.window_size):
                x += [sentence[i]] * (config.window_size*2)
                y += sentence[i-config.window_size:i] + sentence[i+1:i+config.window_size+1]
        
        return x, y

class NegSamplingDataset(W2VDataset):
    """Negative Sampling Dataset.

    Args:
        config (dict): hyperparameters
        word_frequency (dict): word index - word frequency map for negative sampling
        
    """

    def __init__(self, config):
        super().__init__(config)

    def construct_word_idx(self, corpus):
        print('constructing word matrix')
        word_frequency = collections.Counter(itertools.chain.from_iterable(corpus))
        word_frequency = {word: word_frequency[word]**(3/4) for idx, word in enumerate(word_frequency)}
        word_to_idx = {word: idx for idx, word in enumerate(word_frequency)}
        idx_to_word = {word_to_idx[word]: word for word in word_to_idx}
        self.word_frequency = {word_to_idx[word]: word_frequency[word] for word in word_frequency}
        
        return word_to_idx, idx_to_word
    
    def construct_dataset(self, corpus, config):
        print('constructing training dataset')
        target, pos, neg = [], [], []
        for sentence in corpus:
            for i in range(config.window_size, len(sentence) - config.window_size):
                target += [sentence[i]] * (config.window_size*2)
                pos += sentence[i-config.window_size:i] + sentence[i+1:i+config.window_size+1]
                neg.append(self.neg_sample(sentence[i-config.window_size:i+config.window_size+1], config))
        neg = np.array(neg).reshape(-1, config.window_size*2)
        
        return (pos, neg), target
    
    def neg_sample(self, word_contxt, config):
        word_universe = self.idx_to_word.keys() - set(word_contxt)
        word_distn = np.array([self.word_frequency[idx] for idx in word_universe])
        word_distn = word_distn / word_distn.sum()
        
        return np.random.choice(a=list(word_universe), size=config.neg_sample_size*config.window_size*2, p=word_distn)

    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.tolist()
            
        return [self.x[0][idx], self.y[idx], 1], [self.x[1][idx], self.y[idx], 0]

In [5]:
neg_sam_dataset = NegSamplingDataset(config)

constructing word matrix
constructing training dataset


In [5]:
import pickle
# with open('kbanker_nlp_dataset.pkl', 'wb') as f:
#     pickle.dump(nlp_dataset, f)
    
with open('D:\\data\\text\\torch-dataset\\kbanker_nlp_dataset.pkl', 'rb') as f:
    nlp_dataset = pickle.load(f)

In [ ]:
def collate_fn(data):
    seqs, labels = zip(*data)
    return seqs, labels

dataloader = DataLoader(neg_sam_dataset, batch_size=config.batch_size, \
                        shuffle=False, num_workers=0)

target_emb = nn.Embedding(len(neg_sam_dataset), config.embed_dim).float().to(config.device)
context_emb = nn.Embedding(len(neg_sam_dataset), config.embed_dim).float().to(config.device)
target_emb.weight.data.uniform_(-1,1)
context_emb.weight.data.uniform_(-1,1)
sigmoid = nn.Sigmoid()
similar = nn.CosineSimilarity()

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(list(target_emb.parameters()) + list(context_emb.parameters()), lr=5e-3, momentum=0.9)

for epoch in range(500):
    print('Epoch ' + str(epoch))
    for i, sample in enumerate(dataloader):
        pos_idx = sample[0][0].long().to(config.device)
        neg_idx = sample[1][0].long().to(config.device)
        target_idx = sample[0][1].long().to(config.device)
        pos_label = sample[0][2].float().to(config.device)
        neg_label = sample[1][2].float().to(config.device)
        
        pos = context_emb(pos_idx)
        neg = context_emb(neg_idx)
        target = target_emb(target_idx)
        
        pred_pos = sigmoid(similar(pos, target))
        pred_neg = sigmoid(similar(neg, target.unsqueeze(1).expand_as(neg)))

        pos_loss = criterion(pred_pos, pos_label)
        neg_loss = torch.sum(criterion(pred_neg, neg_label.unsqueeze(1).expand_as(pred_neg)))

        pos_loss.retain_grad()
        neg_loss.retain_grad()
        
        loss = pos_loss + neg_loss
        loss.retain_grad()
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        
        if i % 100 == 99:
            print(i, loss.item())
            if i % 100000 == 99:
                with open('./checkpoints/neg_sample_checkpoint_epoch' + str(epoch) + '_' + str(i) + '.pkl', 'wb') as f:
                    pickle.dump(target_emb, f)
                print('./checkpoints/neg_sample_checkpoint_epoch' + str(epoch) + '_' + str(i) + '.pkl saved')

Epoch 0
99 1.4274840354919434
./checkpoints/neg_sample_checkpoint_epoch0_99.pkl saved
199 1.4266541004180908
299 1.4117457866668701
399 1.4178102016448975
499 1.4213440418243408
599 1.422420859336853
699 1.410521388053894
799 1.4329781532287598
899 1.4238032102584839
999 1.4325814247131348
1099 1.427785873413086
1199 1.418020486831665
1299 1.4091848134994507
1399 1.4011914730072021
1499 1.4018125534057617
1599 1.4198497533798218
1699 1.4153056144714355
1799 1.4285775423049927
1899 1.4180374145507812
1999 1.4312057495117188
2099 1.4218358993530273
2199 1.442793369293213
2299 1.4347388744354248
2399 1.4156382083892822
2499 1.4087920188903809
2599 1.4141075611114502
Epoch 1
99 1.426992416381836
./checkpoints/neg_sample_checkpoint_epoch1_99.pkl saved
199 1.4265809059143066
299 1.411516547203064
399 1.417083740234375
499 1.4206494092941284
599 1.4223313331604004
699 1.408756971359253
799 1.4311903715133667
899 1.4232122898101807
999 1.4319427013397217
1099 1.4277105331420898
1199 1.41714417

499 1.411481261253357
599 1.4230128526687622
699 1.401421070098877
799 1.4225894212722778
899 1.4124460220336914
999 1.426813006401062
1099 1.4283559322357178
1199 1.4030455350875854
1299 1.4003541469573975
1399 1.3928070068359375
1499 1.4007227420806885
1599 1.4203131198883057
1699 1.4159891605377197
1799 1.425146222114563
1899 1.4122073650360107
1999 1.4290112257003784
2099 1.410962462425232
2199 1.4258168935775757
2299 1.4335510730743408
2399 1.4107117652893066
2499 1.398393154144287
2599 1.4064035415649414
Epoch 13
99 1.4235801696777344
./checkpoints/neg_sample_checkpoint_epoch13_99.pkl saved
199 1.4245445728302002
299 1.4107062816619873
399 1.4192402362823486
499 1.4108277559280396
599 1.4225919246673584
699 1.4003541469573975
799 1.4218244552612305
899 1.4114875793457031
999 1.4260783195495605
1099 1.4279133081436157
1199 1.402204155921936
1299 1.3995816707611084
1399 1.3923684358596802
1499 1.4004777669906616
1599 1.4201717376708984
1699 1.4159451723098755
1799 1.424801588058471

1099 1.4239299297332764
1199 1.3934950828552246
1299 1.3950456380844116
1399 1.3919074535369873
1499 1.3999524116516113
1599 1.418320894241333
1699 1.4160966873168945
1799 1.419672966003418
1899 1.409827709197998
1999 1.4232912063598633
2099 1.4046841859817505
2199 1.4205100536346436
2299 1.4274259805679321
2399 1.40207839012146
2499 1.388907551765442
2599 1.4037381410598755
Epoch 25
99 1.421613335609436
./checkpoints/neg_sample_checkpoint_epoch25_99.pkl saved
199 1.420454502105713
299 1.406466007232666
399 1.4137810468673706
499 1.4030784368515015
599 1.4181790351867676
699 1.3903248310089111
799 1.4138500690460205
899 1.3965990543365479
999 1.4166507720947266
1099 1.4240434169769287
1199 1.3930941820144653
1299 1.3945589065551758
1399 1.3914291858673096
1499 1.3996973037719727
1599 1.4181259870529175
1699 1.4159162044525146
1799 1.4199070930480957
1899 1.4099854230880737
1999 1.4228609800338745
2099 1.4041752815246582
2199 1.4203684329986572
2299 1.427168607711792
2399 1.400967955589

1599 1.414900541305542
1699 1.4147396087646484
1799 1.4201767444610596
1899 1.4082609415054321
1999 1.4186935424804688
2099 1.3983937501907349
2199 1.4150291681289673
2299 1.4257317781448364
2399 1.3953149318695068
2499 1.3821896314620972
2599 1.4033031463623047
Epoch 37
99 1.4206249713897705
./checkpoints/neg_sample_checkpoint_epoch37_99.pkl saved
199 1.4154770374298096
299 1.4006407260894775
399 1.4065568447113037
499 1.3990633487701416
599 1.41661536693573
699 1.383568525314331
799 1.404003620147705
899 1.3828493356704712
999 1.40566086769104
1099 1.4181911945343018
1199 1.3872089385986328
1299 1.387812614440918
1399 1.385753870010376
1499 1.3975651264190674
1599 1.41412353515625
1699 1.4144560098648071
1799 1.419750690460205
1899 1.4079875946044922
1999 1.4186134338378906
2099 1.3976044654846191
2199 1.4145652055740356
2299 1.425318956375122
2399 1.3948884010314941
2499 1.3816217184066772
2599 1.4030910730361938
Epoch 38
99 1.4202053546905518
./checkpoints/neg_sample_checkpoint_epo

2199 1.4114980697631836
2299 1.4208877086639404
2399 1.3972787857055664
2499 1.375667929649353
2599 1.405508041381836
Epoch 49
99 1.4225778579711914
./checkpoints/neg_sample_checkpoint_epoch49_99.pkl saved
199 1.4105374813079834
299 1.3964884281158447
399 1.4015612602233887
499 1.394449234008789
599 1.4144253730773926
699 1.3779879808425903
799 1.4000033140182495
899 1.383259892463684
999 1.3981573581695557
1099 1.414625644683838
1199 1.3822894096374512
1299 1.401282548904419
1399 1.380066990852356
1499 1.40195894241333
1599 1.4194676876068115
1699 1.4123680591583252
1799 1.414267659187317
1899 1.4033931493759155
1999 1.4181458950042725
2099 1.3873755931854248
2199 1.4109652042388916
2299 1.4204926490783691
2399 1.3974597454071045
2499 1.37516450881958
2599 1.40618896484375
Epoch 50
99 1.4219553470611572
./checkpoints/neg_sample_checkpoint_epoch50_99.pkl saved
199 1.4099621772766113
299 1.3961021900177002
399 1.4017460346221924
499 1.3938307762145996
599 1.4138331413269043
699 1.377071

...

1799 1.3264813423156738
1899 1.3448636531829834
1999 1.3733913898468018
2099 1.344501256942749
2199 1.3238914012908936
2299 1.356119155883789
2399 1.3774819374084473
2499 1.322458267211914
2599 1.3933161497116089
Epoch 244
99 1.3680729866027832
./checkpoints/neg_sample_checkpoint_epoch244_99.pkl saved
199 1.3672082424163818
299 1.324019193649292
399 1.3439083099365234
499 1.319939374923706
599 1.3375145196914673
699 1.2948565483093262
799 1.3560899496078491
899 1.2821965217590332
999 1.2463772296905518
1099 1.361074686050415
1199 1.3280407190322876
1299 1.343066930770874
1399 1.3147008419036865
1499 1.3385697603225708
1599 1.3726606369018555
1699 1.3514151573181152
1799 1.326070785522461
1899 1.3445725440979004
1999 1.3737566471099854
2099 1.3444795608520508
2199 1.3236993551254272
2299 1.3556325435638428
2399 1.3783490657806396
2499 1.3221745491027832
2599 1.3931100368499756
Epoch 245
99 1.3687522411346436
./checkpoints/neg_sample_checkpoint_epoch245_99.pkl saved
199 1.366410017013549

2299 1.3494372367858887
2399 1.3756344318389893
2499 1.3195552825927734
2599 1.3913012742996216
Epoch 256
99 1.3659908771514893
./checkpoints/neg_sample_checkpoint_epoch256_99.pkl saved
199 1.3642477989196777
299 1.3194239139556885
399 1.3407447338104248
499 1.314400553703308
599 1.334686279296875
699 1.2910881042480469
799 1.3552675247192383
899 1.2822163105010986
999 1.2414870262145996
1099 1.3599262237548828
1199 1.3240711688995361
1299 1.339310884475708
1399 1.3124306201934814
1499 1.3367578983306885
1599 1.3709220886230469
1699 1.3461910486221313
1799 1.3214173316955566
1899 1.3391287326812744
1999 1.371288776397705
2099 1.3423385620117188
2199 1.32021164894104
2299 1.3488235473632812
2399 1.3753902912139893
2499 1.3198246955871582
2599 1.3910644054412842
Epoch 257
99 1.365630865097046
./checkpoints/neg_sample_checkpoint_epoch257_99.pkl saved
199 1.3640376329421997
299 1.3190040588378906
399 1.3404810428619385
499 1.3140143156051636
599 1.334409236907959
699 1.2908076047897339
799

199 1.3609367609024048
299 1.3155851364135742
399 1.3376948833465576
499 1.3102624416351318
599 1.3328866958618164
699 1.2887791395187378
799 1.3539612293243408
899 1.2794082164764404
999 1.2382590770721436
1099 1.3581883907318115
1199 1.3208122253417969
1299 1.3362419605255127
1399 1.3099277019500732
1499 1.3336095809936523
1599 1.3684438467025757
1699 1.342862606048584
1799 1.3161554336547852
1899 1.3362147808074951
1999 1.3683173656463623
2099 1.3408986330032349
2199 1.3156185150146484
2299 1.3459386825561523
2399 1.3736231327056885
2499 1.3163583278656006
2599 1.3888466358184814
Epoch 269
99 1.3617053031921387
./checkpoints/neg_sample_checkpoint_epoch269_99.pkl saved
199 1.360695481300354
299 1.3152592182159424
399 1.337449073791504
499 1.3099309206008911
599 1.3326244354248047
699 1.2885581254959106
799 1.3537970781326294
899 1.2787294387817383
999 1.2378920316696167
1099 1.3579802513122559
1199 1.3209501504898071
1299 1.335989236831665
1399 1.309699296951294
1499 1.33343327045440

699 1.287023663520813
799 1.3528984785079956
899 1.2770791053771973
999 1.2331383228302002
1099 1.3548409938812256
1199 1.3173457384109497
1299 1.3334290981292725
1399 1.3073058128356934
1499 1.332585334777832
1599 1.364895224571228
1699 1.3384332656860352
1799 1.3128628730773926
1899 1.3327666521072388
1999 1.3665523529052734
2099 1.339421033859253
2199 1.3119373321533203
2299 1.3429877758026123
2399 1.3709628582000732
2499 1.3123540878295898
2599 1.387235164642334
Epoch 281
99 1.3581902980804443
./checkpoints/neg_sample_checkpoint_epoch281_99.pkl saved
199 1.3568650484085083
299 1.312192440032959
399 1.3363460302352905
499 1.305457353591919
599 1.3295612335205078
699 1.2868225574493408
799 1.3527374267578125
899 1.2768514156341553
999 1.2327919006347656
1099 1.3542616367340088
1199 1.3170981407165527
1299 1.3339641094207764
1399 1.3072519302368164
1499 1.332410454750061
1599 1.364600658416748
1699 1.3381708860397339
1799 1.3127844333648682
1899 1.3324952125549316
1999 1.3669788837432

1199 1.3144190311431885
1299 1.331411600112915
1399 1.3026962280273438
1499 1.3316959142684937
1599 1.3622093200683594
1699 1.33620285987854
1799 1.3079721927642822
